In [1]:
all_files = !ls Raw/*/*txt
path_files = !ls Raw/*/*Path*txt
all_files = set(all_files)
path_files = set(path_files)
all_files = all_files-path_files

In [2]:
f2path = {}
for file in all_files:
    header = file[:file.index('.txt')]
    file = header + '.txt'
    f2path[file] = []
    path = header + '_Annotated_Path.txt'
    possible_paths = [header+f'_Annotated_Path_{i}.txt' for i in range(6)]
    for p in [path] + possible_paths:
        if p in path_files:
            f2path[file].append(p)


In [3]:
import json
jsons = !ls */*/*tile*json
known = set(['P','-','x','\n'])
per_game = {'Raw/Castlevania/CV_tile_spec.json':{'collectable':['H','O','1','2','3','4','5','6','7','F','#','S','G','K','R','>','0','A','U'],
                                             'powerup':['1','F','#','S','5','H','G','O','>','A','U'],
                                             'breakable':['B'],
                                             'solid':['B'],
                                             'hazard':['E'],
                                             'moving':['E'],
                                             'null':['@'],
                                             'portal':['D']},
            'Raw/Kid Icarus - current agent/KI_tile_specs.json':{},
            'Raw/Mario/SMB_tile_specs.json':{'collectable':['o','Q'],
                                         'powerup':['?'],
                                         'hazard':['E','B'],
                                         'moving':['E'],
                                         'breakable':["S"]},
            'Raw/MegaMan/MM_tile_specs.json':{'null': ['@'], 
                                          'solid': ['#', 'B', 'H', 'M', 'C', 'D'], 
                                          'breakable': ['B'], 
                                          'hazard': ['H'], 
                                          'climbable': ['|'], 
                                          'collectable': ['L', 'l', 'W', 'w', '+', '*', 'U'],
                                          'powerup': ['L', 'l', 'W', 'w', '+', '*', 'U'], 
                                          'moving': ['M', 'D'],
                                          'portal':['t']},
            'Raw/Metroid/MT_tile_specs.json':{'breakable': ['B'],
                                          'solid':['B','[',']'],
                                          'portal':['D','(',')',],
                                          'collectable': ['+'],
                                          'powerup': ['+'], 
                                          'null':['@'],
                                          'hazard':['E','[',']'],
                                          'moving':['E']},
            'Raw/Ninja Gaiden/NG_tile_specs.json':{
                                            'hazard':['K','B','E','D','C','M'],
                                            'moving':['K','B','E','D','C','M'],
                                            'collectable': ['F','%',')','*','W','A','1','+','J','T','R'],
                                            'powerup': ['F','%',')','*','W','A','1','+','J','T'], }
}


per_game_annotations = {}

for j in jsons:
    tiles = json.load(open(j))
    tile2annotation = {}
    for tile_type in tiles:        
        for tile in tiles[tile_type]:            
            if tile not in tile2annotation:
                tile2annotation[tile] = []                
            tile2annotation[tile].append(tile_type)
    for tile_type in per_game[j]:        
        for tile in per_game[j][tile_type]:            
            if tile not in tile2annotation:
                tile2annotation[tile] = []                
            tile2annotation[tile].append(tile_type)
    per_game_annotations[j] = tile2annotation
    annotated = set(tile2annotation.keys())
    for level in path_files:
        if level[:10] == j[:10]:
            with open(level) as level_file:
                level_str = level_file.read()
                level_vocab = set(level_str)
                missing = (level_vocab-annotated)-known
                if len(missing) > 0:
                    print(missing)
                    print(j,level)
                    
ignored = set(['passable'])  
combos = set()
for game in per_game_annotations:
    for tile in per_game_annotations[game]:
        affordances = tuple(sorted(set(per_game_annotations[game][tile])-ignored))
        combos.add(affordances)

common_vocab = {('breakable', 'solid'):'S', 
                ('climbable', 'solid'):'|', 
                ('climbable', 'passable','solid'):'|', 
                ('collectable',):'o', 
                ('collectable', 'powerup'):'*', 
                ('collectable', 'solid'):'Q', 
                ('hazard',):'v', 
                ('hazard', 'moving'):'e', 
                ('hazard', 'moving', 'solid'):'E', 
                ('hazard', 'moving', 'passable', 'solid'):'E', 
                ('hazard', 'null', 'solid'):'@', 
                ('hazard', 'solid'):'^', 
                ('moving', 'solid'):'#', 
                ('portal',):'$',  
                ('powerup', 'solid'):'!',
                ('solid',):'X'}


path_vocab = {'S':'Ś', 'E':'Ě', '-':'+', 'e':'ě', 'X':'Ǩ', '*':'★', 
              '|':'¦', '^':'⌅', 'v':'ѷ', '$':'§', 'o':'©'}

start_vocab = {'S':'∫', 'E':'∃', '-':'∓', 'e':'∈', 'X':'∑', '*':'⋇', 
              '|':'≬', '^':'⊤', 'v':'⋎', '$':'∲', 'o':'∘'}

end_vocab = {'S':'√', 'E':'∄', '-':'∔', 'e':'∉', 'X':'∏', '*':'⊛', 
              '|':'∥', '^':'⊥', 'v':'⊻', '$':'∱', 'o':'⊜'}

In [4]:
import itertools
def read_level_file(name):
    raw_level = []
    with open(name) as raw_level_file:
        for line in raw_level_file:
            raw_level.append(line.rstrip())
    return raw_level

width = 32
height = 15
not_found = set()
paths_found = set()

!mkdir processed
!rm processed/*
counter = 0
seen = set()
for file in f2path:   
    json_f = None
    for j in jsons: 
        if file[:10] == j[:10]:
            json_f = j
            break
            
    
    raw_level = read_level_file(file)

    level_height = len(raw_level)   
    level_width = len(raw_level[0])
    
    if level_height < height:
        raw_level = ['-'*level_width]*(height-level_height) + raw_level
    if level_width < width:
        continue
    for path in f2path[file]:
        path_level = read_level_file(path)   
        print(path)
        if level_height < height:
            path_level = ['-'*level_width]*(height-level_height) + path_level
        path_locations = {}
        start_location = None
        for yy_ in range(0,level_height):
            for xx_ in range(level_width):
                if path_level[yy_][xx_] == 'P':
                    start_location = (xx_,yy_)
                    path_locations[(xx_,yy_)] = float('inf')
                if path_level[yy_][xx_] == 'x':
                    path_locations[(xx_,yy_)] = float('inf')
        if not start_location:
            min_x = float('inf')
            for p in path_locations:
                if p[0] < min_x:
                    min_x = p[0]
                    start_location = p
        to_visit = [(start_location,0)]
        visited = set([start_location])
        while len(to_visit) > 0:
            current, val = to_visit.pop(0)
            if current not in path_locations:
                continue      
            if path_locations[current] < val:
                continue
            path_locations[current] = val
            
            for y_offset in [-1,0,1]:
                for x_offset in [-1,0,1]:
                    if y_offset != 0 or x_offset != 0:
                        location = (current[0]+x_offset,current[1]+y_offset)
                        if location not in path_locations:
                            continue
                            
                        if location in path_locations and path_locations[location] < val+1:
                            continue
                        if location in visited:
                            continue
                        visited.add(location)
                        to_visit.append((location,val+1))
        for yy_ in range(0,level_height,height):
            xx_ = 0
            if len(raw_level[yy_:yy_+height]) < height:
                str_chonk ='\n'.join([row[xx_:xx_+width] for row in raw_level[yy_:yy_+height]])
                continue
                
            for xx_ in range(level_width-width):
                
                str_chonk ='\n'.join([row[xx_:xx_+width] for row in raw_level[yy_:yy_+height]])
                path_chonk ='\n'.join([row[xx_:xx_+width] for row in path_level[yy_:yy_+height]])
                locs = []
                for y in range(yy_,yy_+height):
                    for x in range(xx_,xx_+width):
                        locs.append((x,y))
                    locs.append(None)
                        
                
                if '@' in str_chonk or 'x' not in path_chonk:
                    continue
                    
                rows = [row[xx_:xx_+width] for row in path_level[yy_:yy_+height]]
                all_ex = True
                for r in rows[7:11]:
                    if r[12:16] != 'xxxx':
                        all_ex = False
                if all_ex:
                    print(path_chonk)
                    print(path)
                common_chunk = []
                min_l_val = float('inf')
                max_l_val = -float('inf')
                min_l = None
                max_l = None
                for tok,p,l in zip(str_chonk,path_chonk,locs):
                    if l in path_locations:
                        val = path_locations[l]
                        if val < min_l_val:
                            min_l_val = val
                            min_l = l
                        if val > max_l_val:
                            max_l_val = val
                            max_l = l
                for tok,p,l in zip(str_chonk,path_chonk,locs):
                    if tok == '\n':
                        common_chunk.append('\n')
                    else:
                        if tok not in per_game_annotations[json_f]:
                            tok = '-'
                        else:
                            affordances = tuple(sorted(set(per_game_annotations[json_f][tok])))
                            if affordances not in common_vocab:
                                print(tok,file,affordances)
                            tok = common_vocab[affordances]
                        if p == 'x':
                            if l == min_l:
                                common_chunk.append(start_vocab[tok])
                            if l == max_l:
                                common_chunk.append(end_vocab[tok])
                            else:
                                common_chunk.append(path_vocab[tok])
                        else:    
                            common_chunk.append(tok)
                common_chunk = ''.join(common_chunk)
                good = True
                for row in common_chunk.split('\n'):
                    if '$' in row:
                        if row.index('$') > 1 and row.index('$') <= width-1:
                            good = False
                if common_chunk in seen:
                    continue
                seen.add(common_chunk)
                if good:
                    with open(f'processed/{j.split("/")[1]}_{counter}.txt','w') as outfile:
                        counter += 1
                        outfile.write(common_chunk)
                #print(str_chonk)
                #print('\n')
        
    

mkdir: cannot create directory ‘processed’: File exists
Raw/Mario/SuperMarioBros2(J)-World4-3_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World4-1_Annotated_Path.txt
Raw/Castlevania/CV_5_Annotated_Path.txt
Raw/Mario/mario-5-3_Annotated_Path.txt
Raw/Ninja Gaiden/NG_6_Annotated_Path.txt
Raw/Ninja Gaiden/NG_4_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World2-1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World1-2_Annotated_Path.txt
Raw/Ninja Gaiden/NG_7_Annotated_Path.txt
Raw/MegaMan/megaman_1_8_Annotated_Path.txt
Raw/Mario/mario-6-1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-World8-1_Annotated_Path.txt
Raw/Mario/mario-3-1_Annotated_Path.txt
Raw/Metroid/metroid_2_Annotated_Path_1.txt
Raw/Metroid/metroid_2_Annotated_Path_2.txt
Raw/Metroid/metroid_2_Annotated_Path_3.txt
Raw/Metroid/metroid_2_Annotated_Path_4.txt
Raw/Metroid/metroid_2_Annotated_Path_5.txt
Raw/Mario/mario-8-1_Annotated_Path.txt
Raw/Mario/SuperMarioBros2(J)-WorldC-2_Annotated_Path.txt
Raw/Mario/mario-3-3_Annot

In [9]:

common_vocab = {('breakable', 'solid'):'S', 
                ('climbable', 'solid'):'|', 
                ('climbable', 'passable','solid'):'|', 
                ('collectable',):'o', 
                ('collectable', 'powerup'):'*', 
                ('collectable', 'solid'):'Q', 
                ('hazard',):'v', 
                ('hazard', 'moving'):'e', 
                ('hazard', 'moving', 'solid'):'E', 
                ('hazard', 'moving', 'passable', 'solid'):'E', 
                ('hazard', 'null', 'solid'):'@', 
                ('hazard', 'solid'):'^', 
                ('moving', 'solid'):'#', 
                ('portal',):'$',  
                ('powerup', 'solid'):'!',
                ('solid',):'X'}
path_vocab = {'S':'Ś', 'E':'Ě', '-':'+', 'e':'ě', 'X':'Ǩ', '*':'★', 
              '|':'¦', '^':'⌅', 'v':'ѷ', '$':'§', 'o':'©'}

In [10]:
files = !ls processed/*txt

In [11]:
per_game = {}
for f in files:
    game = f.split('_')[0]
    if game not in per_game:
        per_game[game] = []
    per_game[game].append(f)


In [12]:
for game in per_game:
    print(game,len(per_game[game]))

processed/Castlevania 775
processed/Mario 6010
processed/MegaMan 925
processed/Metroid 1898
processed/Ninja Gaiden 920
